In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from __future__ import division
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib.dates import DateFormatter, WeekdayLocator,\
    DayLocator, MONDAY,date2num,num2date,AutoDateLocator
from matplotlib.finance import quotes_historical_yahoo_ohlc, candlestick_ohlc,candlestick2_ochl,volume_overlay3

from stockapp import models as stkmd
from dataapp import models as dtamd
from dataapp import tasks as dtatks
from dataapp import libs as dtalibs
from featureapp import libs as ftlibs
from featureapp import models as ftmd
from stockapp import tasks as stktks
from stockapp import libs as stklibs
import featureapp.models as ftmd
import featureapp.tasks as fttks
import queryapp.models as qrymd
import queryapp.tasks as qrytks

import charts.chartservers.libs as chservlibs
import charts.libs as chlibs

from datascience import models as dtscmd
from datascience import tasks as dtsctks
from datascience import libs as dtsclibs


import featureapp as ftapp
import utility as uty
from utility import models as utymd
import itertools as itt
import multiprocessing as mp
from django.db import connection,connections
from django.db import reset_queries
import time 
import pandas as pd
import numpy as np
import inspect
import imp
import datetime
from talib.abstract import *
import utility.models as utmd
import stockapp.libs as stklib
from utility import codemanager as cdmng
from utility import maintenance as mnt
import os 
import json
from django.contrib.auth.models import AnonymousUser
import threading

stk=stkmd.Stockmeta.objects.get(Symbol='TSLA')
Fromdate=pd.datetime(2008,1,1)
Todate=pd.datetime.today()
Trange=pd.date_range(Fromdate,Todate)
Trange=[T.date() for T in Trange if T.weekday()<=4]

import json
# fttks.computefeatuers(stk.id,Trange)

/usr/local/lib/python2.7/dist-packages/matplotlib/cbook.py:136: MatplotlibDeprecationWarning: The finance module has been deprecated in mpl 2.0 and will be removed in mpl 2.2. Please use the module mpl_finance instead.
  warnings.warn(message, mplDeprecation, stacklevel=1)
/usr/local/lib/python2.7/dist-packages/matplotlib/__init__.py:1401: UserWarning:  This call to matplotlib.use() has no effect
because the backend has already been chosen;
matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)
Using TensorFlow backend.
/usr/local/lib/python2.7/dist-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This 

# Charting

In [ ]:
entries=[
    {'Symbol':'TSLA','TF':pd.datetime(2012,1,1).date(),'T0':pd.datetime(2011,1,1).date()  },
    {'Symbol':'AAPL','TF':pd.datetime(2012,1,1).date(),'T0':pd.datetime(2011,1,1).date()  }
]
chservlibs.request_db_charts(entries,5003)
# img=chlibs.CurrentByFutureChart_bydb(entries[0]['T0'],entries[0]['TF'],entries[0]['Symbol'],indicatorlist=(),pricecols=(),querycols=(),featcols=())


# Running Features

In [ ]:
print "---------------- features-------------------"
featurecodes=ftmd.FeatureComputeCode.objects.all()
computecode=featurecodes[0]
computeclass=computecode.importcomputeclass()
CF=computeclass(stk.id,Trange)
CF.computeall(skipdone=True)
# CF.saveall()
print CF.getfeaturelist()

CF.df=CF.addindicators(CF.df,[
        {'name':'SMAstd','timeperiod':20,'colname':'SMAstd20'},
        {'name':'EMAstd','timeperiod':8,'colname':'EMAstd8'},
    ])

# Running Queries

In [ ]:
querycodes=qrymd.QueryComputeCode.objects.all()
computecode=querycodes[0]
computeclass=computecode.importcomputeclass()
CQ=computeclass(stk.id,Trange)
CQ.computeall(skipdone=True)
CQ.saveall()

In [ ]:
CQ.OutcomeCharts()

In [ ]:
CQ.getquerylist()

In [ ]:
CQ.chartfeatures(addpricecols=(),ip=5562,
addfeatcols=[
    ['CCI5','CCI50'],
    ['PastPROFIT10days','PastLOSS10days'],['FutPROFIT10days','FutLOSS10days']
],
addquerycols=[
    'CCICHERRIES',
]
)

# Registering function

In [ ]:
from datascience import libs as dtsclibs
import pandas as pd

@dtsclibs.register_compfunc(RequiredImports=['import pandas as pd','from dataapp import libs as dtalibs'],overwrite_if_exists=False)
def extractdataset(data_id,Symbol):
    """
    @funcName : test
    @input x : an int
    @output df : pd.DataFrame, some random 2 by 2 
    @description : takes an int and then returns a dummy dataframe. This is just for testing purposes
    @Source : 
    def extractdataset(data_id,Symbol):
        window=60
        window_fut=30
        Tfs=map(lambda x: ( (x.date()-pd.Dateoffset(window)).date(),x.date(), (x.date()+pd.Dateoffset(window_fut)).date() ),
                pd.date_range(start=pd.datetime(2010,1,1),end=pd.datetime.today(),freq='W-MON') )

        N=len(Tfs)
        dfinstants=pd.DataFrame({'T0':map(lambda x: x[0],Tfs),'TF':map(lambda x: x[1],Tfs),'Symbol':[Symbol]*N})
        X,X_Meta=dtalibs.Getbatchdata(dfinstants)

        dfinstants=pd.DataFrame({'T0':map(lambda x: x[1],Tfs),'TF':map(lambda x: x[2],Tfs),'Symbol':[Symbol]*N})
        Y,Y_Meta=dtalibs.Getbatchdata(dfinstants)

        shard=dtscmd.DataShard(Data__id=data_id)
        shard.Info['X_Meta']=X_Meta
        shard.Info['Y_Meta']=Y_Meta
        shard.save()

        np.savez_compressed(shard.shardpath(),X=X,Y=Y)
    """
    
    window=60
    window_fut=30
    Tfs=map(lambda x: ( (x.date()-pd.Dateoffset(window)).date(),x.date(), (x.date()+pd.Dateoffset(window_fut)).date() ),
            pd.date_range(start=pd.datetime(2010,1,1),end=pd.datetime.today(),freq='W-MON') )
    
    N=len(Tfs)
    dfinstants=pd.DataFrame({'T0':map(lambda x: x[0],Tfs),'TF':map(lambda x: x[1],Tfs),'Symbol':[Symbol]*N})
    X,X_Meta=dtalibs.Getbatchdata(dfinstants)
    
    dfinstants=pd.DataFrame({'T0':map(lambda x: x[1],Tfs),'TF':map(lambda x: x[2],Tfs),'Symbol':[Symbol]*N})
    Y,Y_Meta=dtalibs.Getbatchdata(dfinstants)
    
    shard=dtscmd.DataShard(Data__id=data_id)
    shard.Info['X_Meta']=X_Meta
    shard.Info['Y_Meta']=Y_Meta
    shard.save()
    
    np.savez_compressed(shard.shardpath(),X=X,Y=Y)

extractdataset.id


# Creating initial Stock price dataset

In [1]:
from datascience import models as dtscmd
from datascience import tasks as dtsctks
from datascience import libs as dtsclibs

Using TensorFlow backend.
/usr/local/lib/python2.7/dist-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [ ]:
projid,dataid=dtsclibs.register_dataset(project_Name="PredictReturn",project_Info={'description': "Data taken on every Monday. 360 days back and 60 days forward"},
                                        Datatype='RawProcessed',GroupName="AllStocks",tag="1",
                                        data_format='npz',Modeltype='Regression',
                                        TransformedFromDataId=None,TransFuncId=None, use_project_ifexists=True)

dtsctks.CreateStockData_2(360,60,dataid,None)

In [ ]:
print dtscmd.Data.objects.filter(Project__id=7,GroupName="AllStocks",tag="1",Datatype='RawProcessed',
                           Dataformat='npz',Modeltype='Regression')
data = dtscmd.Data.objects.get(Project__id=7,GroupName="AllStocks",tag="1",Datatype='RawProcessed',
                           Dataformat='npz',Modeltype='Regression')

data.datapath()

In [ ]:
dtscmd.DataShard.objects.filter(Data=data).delete()

In [ ]:
dtscmd.DataShard.objects.all()

# Functions on shards

In [ ]:
from datascience import models as dtscmd
from datascience import tasks as dtsctks
from datascience import libs as dtsclibs
from celery import group
import time
import pickle as pkl

@dtsclibs.register_compfunc(Group='test',overwrite_if_exists=True)
def GetMeta(shardId):
    print "----------------------"
    print shardId
    print "----------------------"
    
    shard=dtscmd.DataShard.objects.get(id=shardId)
    X,Y,Meta=shard.getdata()
    p=[ 0,  5,  3,  4, 14, 11,  6,  8, 10,  7,  9, 12, 13,  2,  1]
    X=X[:,:,p]
    

M=dtsctks.applyfunc2data(GetMeta.id,6,wait=True)


In [140]:
dtscmd.DataShard.objects.filter(Data__id=6).delete()

(7940, {u'datascience.DataShard': 7940})

# Creating derived DataSet

In [ ]:
from datascience import models as dtscmd
dtscmd.Project.objects.all()
# data=dtscmd.Data.objects.filter(Project=pr)[0]

In [ ]:
data.delete()

In [3]:
shard=dtscmd.DataShard.objects.get(id=6531)
X,Y,Meta=shard.getdata()

In [70]:
req=list( Meta['MetaX']['columns'] )
print req

['Close', 'Open', 'High', 'Low', 'Volume', 'Symbol', 'SMA10', 'SMA20', 'SMA50', 'SMA100', 'SMA200', 'VolSMA10', 'VolSMA20', 'EMA8', 'EMA20']


In [73]:
d={cc : 1 for cc in Meta['MetaX']['columns']}
curr=sorted( d.keys() )
print curr

['Close', 'EMA20', 'EMA8', 'High', 'Low', 'Open', 'SMA10', 'SMA100', 'SMA20', 'SMA200', 'SMA50', 'Symbol', 'VolSMA10', 'VolSMA20', 'Volume']


In [78]:
p=np.zeros(15)
p

array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.])

In [110]:
p=np.zeros(15)
for i in range(15):
    p[i]=curr.index(req[i])
p=p.astype(int)
p

array([ 0,  5,  3,  4, 14, 11,  6,  8, 10,  7,  9, 12, 13,  2,  1])

In [131]:
X[:,:,p]

array([[[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ..., 
        [84.9, 84.41, 85.03, ..., 433220.0, 85.58218309673974,
         84.17656265613408],
        [83.65, 84.63, 86.03, ..., 410550.0, 85.15280907524203,
         84.12641383174035],
        [82.62, 84.21, 84.62, ..., 413275.0, 84.58996261407715,
         83.98294584776508]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ..., 
        [84.57, 83.42, 85.29, ..., 423885.0, 84.23409763597871,
         83.94719852821734],
        [83.7, 84.35, 84.78, ..., 416885.0, 84.1154092724279,
         83.92365581124426],
        [83.76, 84.11, 84.48, ..., 409760.0, 84.03642943411059,
         83.90806954350671]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        

In [120]:
H[:,p]

array([83.76, 84.11, 84.48, 83.62, 181800.0, u'ESS', 84.24000000000011,
       84.70999999999984, 81.70200000000004, 79.3874999999999,
       72.04099999999991, 381550.0, 409760.0, 84.03642943411059,
       83.90806954350671], dtype=object)

In [104]:
H[:,:][-1]

array([83.76, 83.90806954350671, 84.03642943411059, 84.48, 83.62, 84.11,
       84.24000000000011, 79.3874999999999, 84.70999999999984,
       72.04099999999991, 81.70200000000004, u'ESS', 381550.0, 409760.0,
       181800.0], dtype=object)

In [101]:
H[:,p][-1]

array([83.76, 181800.0, 409760.0, 84.03642943411059, 84.48,
       83.90806954350671, 84.24000000000011, 72.04099999999991,
       79.3874999999999, 81.70200000000004, 84.70999999999984, 84.11,
       u'ESS', 381550.0, 83.62], dtype=object)

In [58]:
X[1,-1,:]

array([83.76, 83.90806954350671, 84.03642943411059, 84.48, 83.62, 84.11,
       84.24000000000011, 79.3874999999999, 84.70999999999984,
       72.04099999999991, 81.70200000000004, u'ESS', 381550.0, 409760.0,
       181800.0], dtype=object)

In [32]:
np.isnan(X[1,:,:])

TypeError: ufunc 'isnan' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''

In [5]:
Y.shape

(394, 60, 15)

In [28]:
Z=np.stack((X.transpose((0,2,1)),Y.transpose((0,2,1))),axis=1)

ValueError: all input arrays must have the same shape

In [136]:
Y[1,0,:]

array([83.76, 83.90806954350671, 84.03642943411059, 84.48, 83.62, 84.11,
       84.24000000000011, 79.3874999999999, 84.70999999999984,
       72.04099999999991, 81.70200000000004, u'ESS', 381550.0, 409760.0,
       181800.0], dtype=object)